In [4]:
import pandas as pd
import numpy as np
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Plotly Express


In [5]:
import plotly.express as px

In [6]:
df_s = pd.read_csv('南棟電梯.csv',index_col = 0)  
print(df_s.tail())

     hour floor direction  count
860    19    B4        up    121
861    20    B4        up     82
862    21    B4        up     81
863    22    B4        up     13
864    23    B4        up     34


plotly express 有點像是 ggplot in R，<br>
只要指定x,y,color等就能自動幫你分類，<br>
下面的圖表以南棟電梯的外部電梯呼叫資料為例，<br>
x軸為樓層、以方向分開並上色，最後以時間(小時)作為動畫影格。

In [7]:
floor_list = ['B4','B3', 'B2','B1','01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15']

fig = px.bar(data_frame=df_s, 
             
             ## 這裡的data frame都先用 R aggregate過了，以節省運算時間
             x = "floor", y = "count",
             
             ## 以方向分兩張圖，並上色
             facet_col="direction",color="direction",
             
             ## 自動生成slider
             animation_frame="hour",
             
             
             labels={"count":"Frequency",
                     "hour":"Local Time"})

# X axis
fig.update_xaxes(nticks=len(floor_list), ##指定breaks的數量
                 
                 ## 手動將X的類別排序
                 type="category",
                 categoryarray=floor_list)

# Y axis
fig.update_yaxes(range=[0, 1000])

# Layout
fig.update_layout(title_text='Frequency of Elevator Calls')

fig.show()

## Data

In [6]:
import plotly.graph_objects as go
from ipywidgets import widgets

這裡將不同大樓資料混在一起，最後才能依大樓名稱篩選

In [16]:
## 檔案名(註:資料是arregate後的table)

files_list = ['研究大樓','北棟病床梯','南棟電梯']

df = pd.DataFrame()

for i in files_list:
    data = pd.read_csv(i+'.csv', index_col = 0)
    data['building'] = i
    print(i)
    print(data.shape)
    
    df = df.append(data, ignore_index = True) 

## 請檢查表格的數量是否正確 有些表格有缺失(e.g.北棟客梯)
    
print(df.shape)
print(df.head())

研究大樓
(864, 5)
北棟病床梯
(864, 5)
南棟電梯
(864, 5)
(2592, 5)
   hour floor direction  count building
0     0    01      down      4     研究大樓
1     1    01      down      2     研究大樓
2     2    01      down      2     研究大樓
3     3    01      down      4     研究大樓
4     4    01      down      1     研究大樓


## Dash

In [14]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash()
app.layout = html.Div([
    html.H1('Outer calls'),
    dcc.Dropdown(
        id="building",
        options=[
            {'label': '研究大樓', 'value': '研究大樓'},
            {'label': '北棟病床梯', 'value': '北棟病床梯'},
            {'label': '南棟電梯', 'value': '南棟電梯'}
        ],
        value='研究大樓'
    ),
    dcc.Graph(id="graph")
])

@app.callback(
    Output('graph', 'figure'),
    [Input('building', 'value')])
def update_figure(selected_building):
    
    fig = px.bar(data_frame=df[df.building == selected_building],
                 x = "floor", y = "count",
                 facet_col="direction",color="direction",
                 animation_frame="hour",
                 range_y=[0, 1000],
                 labels={"count":"Frequency",
                         "hour":"Local Time"})
    # X axis
    fig.update_xaxes(nticks=len(floor_list),
                     type="category",
                    categoryarray=floor_list)
    
    fig.update_layout(title_text='Frequency of Elevator Calls in ' + selected_building)
    
    return fig

app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 990-974-041
Debugger PIN: 990-974-041
Debugger PIN: 990-974-041
Debugger PIN: 990-974-041
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
